In [ ]:
import numpy as np

In [ ]:
import optuna

# Getting started

In [ ]:
from functions import black_hole

In [ ]:
black_hole?

In [ ]:
def objective(trial):
    x = trial.suggest_uniform('x',-10,10)
    y = trial.suggest_uniform('y',-10,10)
    return black_hole(x,y)

In [ ]:
study = optuna.create_study(sampler=optuna.samplers.CmaEsSampler())
study.optimize(objective, n_trials=50)

In [ ]:
study.best_params

In [ ]:
study.best_value

In [ ]:
black_hole(**study.best_params)

In [ ]:
study.trials_dataframe().tail()

In [ ]:
from optuna.visualization import plot_contour, plot_optimization_history

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_contour(study)

# Sklearn

In [ ]:
from sklearn.datasets import make_friedman1
from sklearn.model_selection import train_test_split

X, y = make_friedman1(n_samples=1000, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor() # default hyper-parameters
rf.fit(X_train, y_train)
#rf.score(X_val, y_val)
y_test_pred = rf.predict(X_test)
np.sqrt(mean_squared_error(y_test_pred, y_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

def rf_objective(trial):
    rf = RandomForestRegressor(n_estimators=trial.suggest_categorical('n_estimators',[10,50, 100, 500]),
                               max_depth=trial.suggest_int('max_depth',2, 20),
                               min_samples_split=trial.suggest_int('min_samples_split',2, 10))

    score = cross_val_score(rf, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error').mean()

    return score

In [ ]:
rf_study = optuna.create_study(direction="maximize")
rf_study.optimize(rf_objective, n_trials=10)

In [ ]:
rf_study.best_value

In [ ]:
rf_study.best_params

... let's do some more iterations

In [ ]:
rf_study.optimize(rf_objective, n_trials=20)

In [ ]:
plot_optimization_history(rf_study)

In [ ]:
from optuna.visualization import plot_param_importances

plot_param_importances(rf_study)

In [ ]:
from optuna.visualization import plot_contour

plot_contour(rf_study,params=['max_depth','n_estimators'])